In [44]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("../../../bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/gathering_affinities.ipynb")

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


###Patient tumors

In [2]:
fs = [x for x in os.listdir('/cellar/data/TCGA/Firehose__2015_06_01/CNV') if x[:4] != 'TCGA']
all_dfs = []
for f in fs:
    df = pd.read_csv('/cellar/data/TCGA/Firehose__2015_06_01/CNV/{0}/all_data_by_genes.txt'.format(f),
                sep='\t')
    df.index = df['Gene Symbol']
    df = df.drop(['Gene Symbol', 'Locus ID', 'Cytoband'], axis=1)
    all_dfs.append(df.transpose())

In [40]:
df = pd.concat(all_dfs)

In [27]:
# get patients we are using
patient_mutations = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.TCGA.inclusive.mut.csv',
                                index_col=0)

In [41]:
patients = list(patient_mutations.index)

In [42]:
def get_barcode(x):
    return x[:12]
df.index = pd.Series(df.index).apply(get_barcode)

In [43]:
df = df[~df.index.duplicated(keep='first')]

In [44]:
df.shape

(10844, 24776)

In [45]:
df.head()

,7SK|ENSG00000232512.2,7SK|ENSG00000249352.3,7SK|ENSG00000254144.2,7SK|ENSG00000260682.2,7SK|ENSG00000271765.1,7SK|ENSG00000271814.1,7SK|ENSG00000271818.1,A1BG,A1CF,A2M,...,snoZ185,snoZ247,snoZ278,snoZ40,snoZ5,snoZ6|ENSG00000252200.1,snoZ6|ENSG00000253067.1,snoZ6|ENSG00000264452.1,snoZ6|ENSG00000266692.1,snosnR66
TCGA-2K-A9WE,0.202,0.229,-0.017,0.221,-0.032,0.073,0.000,-0.019,-0.017,0.229,...,0.386,0.011,-0.009,0.209,0.011,-0.019,0.232,0.202,0.202,-0.005
TCGA-2Z-A9J1,-0.026,-0.004,-0.014,0.680,0.000,0.146,0.713,-0.056,-0.022,0.663,...,0.740,0.008,-0.043,0.018,0.008,-0.056,-0.004,-0.040,-0.040,-0.014
TCGA-2Z-A9J2,-0.028,-0.019,0.634,0.016,-0.025,-0.352,-0.007,-0.006,0.000,0.673,...,-0.008,0.004,-0.050,0.700,0.004,-0.006,-0.019,-0.028,-0.028,-0.003
TCGA-2Z-A9J3,0.004,-0.021,0.857,0.945,-0.027,0.200,-0.016,0.015,-0.006,-0.002,...,2.497,0.004,0.016,-0.049,0.004,0.015,-0.021,0.004,0.004,0.002
TCGA-2Z-A9J5,-0.002,-0.017,-0.021,0.642,-0.014,0.153,-0.003,-0.011,0.000,0.629,...,0.627,0.485,0.002,0.616,0.485,-0.011,-0.017,-0.002,-0.002,-0.010


In [48]:
df = df.ix[[x for x in patients if x in list(df.index)]]

In [49]:
df.shape

(7027, 24776)

In [50]:
df.to_csv('/cellar/users/ramarty/Data/hla_ii/clinical/CNV.TCGA.gistic2.csv')

###Tissue type peaks; find overlapping

In [23]:
tissues_to_accept = ['GBM','LUAD','LUSC','BLCA','PAAD','COAD','STAD','SKCM'] +  \
                    ['THCA','HNSC','READ','LGG', 'BRCA','OV','PRAD']

In [45]:
ensemble = gather_protein_sequences_all()
def check_gene(gene):
    if gene in list(ensemble.gene):
        return True
    else:
        return False

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [67]:
# deleted
fs = [x for x in os.listdir('/cellar/data/TCGA/Firehose__2015_06_01/CNV') if x[:4] != 'TCGA']
tissue_peak_dict = {}
for f in fs:
    tissue = f.split('_')[1].split('-')[0]
    if tissue in tissues_to_accept:
        df = pd.read_csv('/cellar/data/TCGA/Firehose__2015_06_01/CNV/{0}/table_del.conf_99.txt'.format(f),
                    sep='\t')
        string_of_genes = ''
        for number, genes in zip(list(df.n_genes_in_peak), list(df.genes_in_peak)):
            if number > 0:
                string_of_genes += genes
        tissue_peak_dict[tissue] = string_of_genes.split(',')[:-1]
all_genes = []
for tissue in tissue_peak_dict.keys():
    all_genes.extend(tissue_peak_dict[tissue])
print len(all_genes)
all_genes = list(set(all_genes))
print len(all_genes)
gene_dictionary = {}
for gene in all_genes:
    count = 0
    for tissue in tissue_peak_dict.keys():
        if gene in tissue_peak_dict[tissue]:
            count += 1
    gene_dictionary[gene] = count
cnv_count = pd.DataFrame.from_dict(gene_dictionary, orient='index').reset_index()
cnv_count.columns = ['gene', 'cnv']
amped_genes = []
for gene in cnv_count.sort_values('cnv', ascending=False).gene[:30]:
    if check_gene(gene):
        amped_genes.append(gene)
print len(amped_genes)
with open('/cellar/users/ramarty/Data/hla_ii/generated_data/mutations/deleted_genes.txt', 'w') as f:
    for gene in amped_genes[:10]:
        f.write('{0}\n'.format(gene))

4881
3750
12


In [68]:
# amplified
fs = [x for x in os.listdir('/cellar/data/TCGA/Firehose__2015_06_01/CNV') if x[:4] != 'TCGA']
tissue_peak_dict = {}
for f in fs:
    tissue = f.split('_')[1].split('-')[0]
    if tissue in tissues_to_accept:
        df = pd.read_csv('/cellar/data/TCGA/Firehose__2015_06_01/CNV/{0}/table_amp.conf_99.txt'.format(f),
                    sep='\t')
        string_of_genes = ''
        for number, genes in zip(list(df.n_genes_in_peak), list(df.genes_in_peak)):
            if number > 0:
                string_of_genes += genes
        tissue_peak_dict[tissue] = string_of_genes.split(',')[:-1]
all_genes = []
for tissue in tissue_peak_dict.keys():
    all_genes.extend(tissue_peak_dict[tissue])
print len(all_genes)
all_genes = list(set(all_genes))
print len(all_genes)
gene_dictionary = {}
for gene in all_genes:
    count = 0
    for tissue in tissue_peak_dict.keys():
        if gene in tissue_peak_dict[tissue]:
            count += 1
    gene_dictionary[gene] = count
cnv_count = pd.DataFrame.from_dict(gene_dictionary, orient='index').reset_index()
cnv_count.columns = ['gene', 'cnv']
amped_genes = []
for gene in cnv_count.sort_values('cnv', ascending=False).gene[:30]:
    if check_gene(gene):
        amped_genes.append(gene)
print len(amped_genes)
with open('/cellar/users/ramarty/Data/hla_ii/generated_data/mutations/amplified_genes.txt', 'w') as f:
    for gene in amped_genes[:10]:
        f.write('{0}\n'.format(gene))

989
796
26
